In [ ]:
#install
import pybamm
import pybammeis
import matplotlib.pyplot as plt
import numpy as np

# Set seed 
np.random.seed(0)

# Define the DFN model or PMe model with degradation mechanisms(SEI growth & lithium plating and particle cracking)

model = pybamm.lithium_ion.DFN(
    {
      "thermal": "isothermal",
      "SEI": "solvent-diffusion limited",
       "SEI porosity change": "true",
       "lithium plating": "partially reversible",
       "lithium plating porosity change": "true",  # alias for "SEI porosity change"
       "particle mechanics": ("swelling and cracking","swelling only"),
       "SEI on cracks": "true",
        "loss of active material": "stress-driven",
        "calculate discharge energy": "true",  # for compatibility with older PyBaMM versionsg
        
    }
)

# Load parameter values (OKane2022)
param = pybamm.ParameterValues("OKane2022")
param.update({
    "Initial temperature [K]": 278.15, #change the temperature 
    "SEI growth activation energy [J.mol-1]": 37000.0,
    #"Negative electrode initial crack length [m]":2e-99,
    #"Positive electrode initial crack length [m]":2e-8,
    #"Negative electrode initial crack width [m]":1.5e-99,
    #"Positive electrode initial crack width [m]":1.5e-8,
    "Ambient temperature [K]": 278.15,},)

# Discretization points
var_pts = {
    "x_n": 5,
    "x_s": 5,
    "x_p": 5,
    "r_n": 30,
    "r_p": 30,
}

# Solver
safe_solver = pybamm.IDAKLUSolver() #for fast simulation
#safe_solver = pybamm.CasadiSolver() #for accurate simulation
#pybamm.set_logging_level("NOTICE")

# Number of cycles
cycle_number = 1000  # for  aging
eis_cycles = [1, 100, 300, 400, 500, 700, 900, 1000]  #  EIS only at these cycles
# Frequencies for EIS
frequencies = np.logspace(-4, 3, 30)# (10^(-4) to 10^(3))

# Impedance storage
impedance_data = {}
aged_model = model
all_solutions = [] 
# Run 1 full charge/discharge/hold cycle at a time
for cycle in range(1, cycle_number + 1):
    print(f"\n Running cycle {cycle}...")

    # Define a single full cycle
    exp = pybamm.Experiment([
        (
            "Discharge at 1C until 2.5 V",
            "Charge at 5 A until 4.2 V",
            "Hold at 4.2 V until C/100",
            "Rest for 30 minutes",
        )
    ],period="30 minutes",) #change this for good accuracy

    # Run the cycle using the aged model 
    sim = pybamm.Simulation(
        aged_model,
        parameter_values=param,
        experiment=exp,
        solver=safe_solver,
        var_pts=var_pts,
    )
    sol = sim.solve(calc_esoh=True,initial_soc=1)
    all_solutions.append(sol)
    # Calculate the capacity and SOH
    capacity = sol.summary_variables["Capacity [A.h]"]
    if cycle is  1:
        initial_Q=sol.summary_variables["Capacity [A.h]"]    
    SOH = capacity[-1] / initial_Q[0]
 
    print(f"SOH: {SOH:.8f} ")
    # Update the model with the aged state
    aged_model = model.set_initial_conditions_from(sol)

    # Run EIS using the aged model
    if cycle in eis_cycles:
        print(f" Running EIS for cycle {cycle}")
        eis_sim = pybammeis.EISSimulation(
            aged_model,
            parameter_values=param,
            var_pts=var_pts
        )
    eis_sim.solve(frequencies)
    impedance_data[cycle] = eis_sim.solution
    


# Plot Nyquist curves for each cycle

plt.figure(figsize=(10, 8))
for cycle, Z in impedance_data.items():
    plt.plot(Z.real, -Z.imag, marker='o' )

plt.title("EIS Nyquist Plot per Cycle")
plt.xlabel("Real(Z) [Ohm]")
plt.ylabel("-Imag(Z) [Ohm]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pybamm

# Load model
model = pybamm.lithium_ion.SPM()

# Use Chen2020 parameters
param = pybamm.ParameterValues("Chen2020")

# Define standard CC-CV charging
standard_experiment = pybamm.Experiment([
    (
    "Charge at 5 A until 4.2 V",
    "Hold at 4.2 V until C/20",
    )
])

# Define custom multi-stage charging
custom_experiment = pybamm.Experiment([
(
    "Discharge at 1C until 2.5 V",
    "Charge at 5 A until 4.2 V",
    "Hold at 4.2 V until C/100",
    "Rest for 30 minutes",
)
])

# Run standard strategy
sim_standard = pybamm.Simulation(model, parameter_values=param, experiment=standard_experiment)
sim_standard.solve()

# Run custom strategy
sim_custom = pybamm.Simulation(model, parameter_values=param, experiment=custom_experiment)
sim_custom.solve()

# Plot both results
sim_standard.plot()
sim_custom.plot()


2025-05-03 09:39:52.401 - [WARNING] base_model.default_solver(417): The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.
2025-05-03 09:39:53.624 - [WARNING] base_model.default_solver(417): The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.


interactive(children=(FloatSlider(value=0.0, description='t', max=83.39531778449366, step=0.8339531778449366),…

interactive(children=(FloatSlider(value=0.0, description='t', max=3.4370512279961285, step=0.03437051227996128…

In [2]:
from math import floor, log10
def file_name_with_zero(n):
    m = floor(log10(n))
    s = ['0' for i in range (4-m)]
    return f"{''.join(s)}{n}"

In [27]:
import pandas as pd

data = [pd.read_csv(f"dataset/cleaned_dataset/data/{file_name_with_zero(i)}.csv") for i in range (1, 7566)]

In [ ]:
from matplotlib import pyplot as plt

for i in range (0, 100, 10):
    print(f"- {i} -")
    print(data[i].head())
    print("- - -")


- 0 -
- charge -
   Voltage_measured  Current_measured  Temperature_measured  Current_load  \
0          4.246711          0.000252              6.212696        0.0002   
1          4.246764         -0.001411              6.234019        0.0002   
2          4.039277         -0.995093              6.250255        1.0000   
3          4.019506         -0.996731              6.302176        1.0000   
4          4.004763         -0.992845              6.361645        1.0000   

   Voltage_load    Time  
0         0.000   0.000  
1         4.262   9.360  
2         3.465  23.281  
3         3.451  36.406  
4         3.438  49.625  
- discharge -
   Voltage_measured  Current_measured  Temperature_measured  Current_charge  \
0          3.486189          0.001417              5.990142         -0.0006   
1          3.746592          1.489057              5.992528          1.4995   
2          3.775438          1.490426              5.991340          1.4995   
3          3.794844          1.491